In [94]:
from __future__ import annotations
from pathlib import Path
import yaml
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import torch

from data_management.dataset_file import Goal

from goal_evaluation.goal_example import GoalExample
from goal_evaluation.goal_data import SEP_ID, START_ID, tokenize_example
from transformers import OPTForCausalLM, GPT2Tokenizer 
from util.constants import TRAINING_CONF_NAME 




In [95]:

class GoalWrapper:
    def __init__(self, goal_model: OPTForCausalLM, tokenizer: GPT2Tokenizer, max_seq_len: int):
        self.goal_model = goal_model
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.sft = torch.nn.Softmax()
    
    def get_score_and_n_left(self, example: GoalExample) -> tuple[float, float]:
        tokenized_input = tokenize_example(self.tokenizer, self.max_seq_len, example) 
        query = tokenized_input["input_ids"] + [SEP_ID]
        input_tensor = torch.tensor([query], device=self.goal_model.device)
        attn_mask = torch.tensor([[1 for _ in range(len(query))]], device=self.goal_model.device)
        model_inputs = {
            "input_ids": input_tensor,
            "attention_mask": attn_mask,
        }
        with torch.no_grad():
            output = self.goal_model(**model_inputs)
        n_step_logits = output.logits[0, -1, START_ID:]
        probs = self.sft(n_step_logits)
        exp_n_steps = probs @ torch.arange(len(probs), device=self.goal_model.device, dtype=torch.float32)

        loss_inputs = input_tensor[:, :-1]
        loss_output_logits = output.logits[:, :-1]
        shift_logits = loss_output_logits[0, :-1, :].contiguous()
        shift_logprobs = torch.log_softmax(shift_logits, dim=-1)
        shift_labels = loss_inputs[0, None, 1:].contiguous()
        scores = torch.gather(shift_logprobs, 1, shift_labels)
        return float(exp_n_steps), float(scores.mean())

    @classmethod
    def from_checkpoint(cls, checkpoint_loc: Path) -> GoalWrapper:
        model_training_conf_loc = checkpoint_loc.parents[0] / TRAINING_CONF_NAME
        with model_training_conf_loc.open("r") as fin:
            training_conf = yaml.load(fin, Loader=yaml.Loader)
        model = OPTForCausalLM.from_pretrained(checkpoint_loc)
        model = model.cuda()
        tokenizer = GPT2Tokenizer.from_pretrained(training_conf["model_name"], truncation_side="left")
        print(tokenizer.truncation_side)
        max_seq_len = training_conf["max_seq_len"]
        return cls(model, tokenizer, max_seq_len)




In [96]:
checkpoint_loc = Path("/home/ubuntu/coq-modeling/models/goal/checkpoint-1000")
w = GoalWrapper.from_checkpoint(checkpoint_loc)

left


In [97]:
e1 = GoalExample("x, y: R\nHxy: (x <= y)%R\n\n(IZR (Zfloor x) <= y)%R", 4)
e2 = GoalExample("", 1)


e3 = GoalExample("A: Type\nAe: Equiv A\nAplus: Plus A\nAmult: Mult A\nAzero: Zero A\nAone: One A\nAnegate: Negate A\nAdec_recip: DecRecip A\nH: DecField A\nH0: \u2200 x0 y : A, Decision (x0 = y)\nB: Type\ne: Equiv B\nplus0: Plus B\nmult0: Mult B\nzero0: Zero B\none0: One B\nnegate0: Negate B\nU: IntegersToRing B\nH1: Integers B\nipw: Pow A B\nIntPowSpec0: IntPowSpec A B ipw\nx: A\nn: B\nEx: x = 0\n\n0 ^ (- n) = / 0 ^ n<G>A: Type\nAe: Equiv A\nAplus: Plus A\nAmult: Mult A\nAzero: Zero A\nAone: One A\nAnegate: Negate A\nAdec_recip: DecRecip A\nH: DecField A\nH0: \u2200 x0 y : A, Decision (x0 = y)\nB: Type\ne: Equiv B\nplus0: Plus B\nmult0: Mult B\nzero0: Zero B\none0: One B\nnegate0: Negate B\nU: IntegersToRing B\nH1: Integers B\nipw: Pow A B\nIntPowSpec0: IntPowSpec A B ipw\nx: A\nn: B\nEx: x \u2260 0\n\nx ^ (- n) = / x ^ n", 18)
e4 = GoalExample("A: Type\nAe: Equiv A\nAplus: Plus A\nAmult: Mult A\nAzero: Zero A\nAone: One A\nAnegate: Negate A\nAdec_recip: DecRecip A\nH: DecField A\nH0: \u2200 x0 y : A, Decision (x0 = y)\nB: Type\ne: Equiv B\nplus0: Plus B\nmult0: Mult B\nzero0: Zero B\none0: One B\nnegate0: Negate B\nU: IntegersToRing B\nH1: Integers B\nipw: Pow A B\nIntPowSpec0: IntPowSpec A B ipw\nx: A\nEx: x \u2260 0\n\n\u2200 n : B, x ^ (- n) = / x ^ n \u2194 x ^ (- (1 + n)) = / x ^ (1 + n)", 9)

e5 = GoalExample("l: list nat\n\nl <> nil -> exists h : nat, min l = Some h", 10)
e6 = GoalExample("n: nat\nl: list nat\n\nn :: l <> nil -> exists h : nat, min (n :: l) = Some h", 10)

e7 = GoalExample("hsfkldfh;salhfaksld;fhalsdhfsshkfla;sdfhkjal;sdfhkal;sdhkl", 10)

#e6 = GoalExample("")


In [105]:
e, l = w.get_score_and_n_left(e7)
e, l

(20.69045639038086, -10.862499237060547)

In [99]:
sft = torch.nn.Softmax()
log_of_interest = l.logits[0, -1, (START_ID + 1):]
log_of_interest[:10]

AttributeError: 'float' object has no attribute 'logits'

In [ ]:
torch.argmax(log_of_interest) + 1

tensor(2, device='cuda:0')